# Asset Data
### TODO
- create ipfs address of data folder
- create contract code, initiate contracts in 3 networks and upload data to these contracts

In [1]:
import json
import os
import ast
import pandas as pd
from warnings import filterwarnings
from tqdm.notebook import tqdm
from jsonschema import validate

from src.lcd_extractor import extract_assets
from src.chain_registry_extractor import get_chain_names_and_lcd_dicts
from src.json_export import get_asset_json_dict

filterwarnings('ignore')
tqdm.pandas()

EXTRACT_DATA = False

### Get chain names and lcd dict from chain-registry repo

In [2]:
chain_id_name_dict, chain_id_lcd_dict, _ = get_chain_names_and_lcd_dicts()

no lcd api for microtick


### Get asset data from nodes

In [3]:
assets_df = pd.DataFrame(columns=['denom', 'supply', 'denom_base', 'path', 'channels', 'one_channel',
                                  'chain_id_counterparty', 'channel_id_counterparty', 'type_asset',
                                  'type_asset_base', 'chain_id'])
if EXTRACT_DATA:
    for chain_id, node_lcd_url_list in list(chain_id_lcd_dict.items()):
        print(chain_id)
        asset_df = extract_assets(chain_id, node_lcd_url_list)
        if asset_df is None:
            print(f'data has not been loaded for {chain_id}, lcd apis not work')
            continue
        asset_df.to_csv(f'data_csv/assets_{chain_id}.csv')
        assets_df = pd.concat([assets_df, asset_df])
else:
    asset_raw_file_names = [f.path for f in os.scandir('data_csv')
                            if not f.is_dir() if
                            f.path.split('/')[1][0] not in ('_', '.') and f.path.split('/')[1] != 'all_assets.csv']
    for asset_raw_file_name in asset_raw_file_names:
        asset_df = pd.read_csv(asset_raw_file_name)
        asset_df = asset_df.drop_duplicates()
        if 'channels' in asset_df.columns:
            asset_df['channels'] = asset_df.channels.map(lambda x: ast.literal_eval(x) if type(x) == str else None)
        if 'denom_units' in asset_df.columns:
            asset_df['denom_units'] = asset_df.denom_units.map(
                lambda x: ast.literal_eval(x) if type(x) == str else None)
        assets_df = pd.concat([assets_df, asset_df])

In [4]:
assets_one_channel_df = \
    assets_df[assets_df.one_channel == True][['denom', 'supply', 'chain_id', 'denom_base', 'path', 'channels',
                                              'chain_id_counterparty', 'channel_id_counterparty', 'type_asset',
                                              'one_channel', 'type_asset_base']].merge(
        assets_df[['denom', 'supply', 'chain_id', 'description', 'denom_units', 'display', 'name', 'symbol']].rename(
            columns={'supply': 'supply_base', 'chain_id': 'chain_id_base', 'denom': 'denom_base'}),
        how='left',
        left_on=['chain_id_counterparty', 'denom_base'],
        right_on=['chain_id_base', 'denom_base']
    )
assets_not_one_channel_df = assets_df[assets_df.one_channel != True]
assets_not_one_channel_df['supply_base'] = assets_not_one_channel_df.apply(
    lambda _row: _row['supply'] if _row['one_channel'] is None else None, axis=1)
assets_not_one_channel_df['chain_id_base'] = assets_not_one_channel_df.apply(
    lambda _row: _row['chain_id'] if _row['one_channel'] == True else None, axis=1)

assets_df = pd.concat([
    assets_one_channel_df,
    assets_not_one_channel_df]).reset_index(drop=True)[[
        'chain_id', 'denom', 'type_asset', 'supply', 'description', 'denom_units', 'display', 'name', 'symbol', 'uri',
        'denom_base', 'type_asset_base', 'path', 'channels', 'chain_id_counterparty', 'channel_id_counterparty',
        'supply_base', 'chain_id_base', 'one_channel']]

assets_df = assets_df.fillna(value={'description': '', 'denom_units': '', 'display': '', 'name': '', 'symbol': ''})
assets_df.to_csv('data_csv/all_assets.csv')
assets_df

,chain_id,denom,type_asset,supply,description,denom_units,display,name,symbol,uri,denom_base,type_asset_base,path,channels,chain_id_counterparty,channel_id_counterparty,supply_base,chain_id_base,one_channel
0,loyal-main-02,ibc/0471F1C4E7AFD3F07702BEF6DC365268D64570F7C1...,ics20,100000,,,,,,NaN,uosmo,sdk.coin,transfer/channel-1,[channel-1],osmo-test-4,channel-2463,NaN,NaN,True
1,loyal-main-02,ibc/C0E66D1C81D8AAF0E6896E05190FDFBC222367148F...,ics20,1000000,,,,,,NaN,uaxl,sdk.coin,transfer/channel-2,[channel-2],axelar-testnet-lisbon-3,channel-165,NaN,NaN,True
2,dimension_37-1,ibc/2CC0B1B7A981ACC74854717F221008484603BB8360...,ics20,600098,,,,,,NaN,uaxl,sdk.coin,transfer/channel-0,[channel-0],axelar-dojo-1,channel-49,1066932465248493,axelar-dojo-1,True
3,dimension_37-1,ibc/6771714762A6F8F541DE7C461F09C68F310621315F...,ics20,93300000000000000,,,,,,NaN,wavax-wei,sdk.coin,transfer/channel-0,[channel-0],axelar-dojo-1,channel-49,10927697941851456549554,axelar-dojo-1,True
4,dimension_37-1,ibc/8E27BA2D5493AF5636760E354E46004562C46AB7EC...,ics20,272785050189,,,,,,NaN,uusdc,sdk.coin,transfer/channel-0,[channel-0],axelar-dojo-1,channel-49,46015284903012,axelar-dojo-1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10592,sentinelhub-2,ibc/7964F196E3F96B15675F6A6FBF1878B16B30FBB46F...,ics20,40000000,,,,,,NaN,udvpn,sdk.coin,transfer/channel-14/transfer/channel-2,"[channel-14, channel-2]",osmosis-1,channel-21,None,None,False
10593,sentinelhub-2,ibc/98FF2F49F25A44E724DB3E4E7521F7899449D2211C...,ics20,100000,,,,,,NaN,udvpn,sdk.coin,transfer/channel-69/transfer/channel-1,"[channel-69, channel-1]",sifchain-1,channel-88,None,None,False
10594,sentinelhub-2,ibc/C6FB13D55E3A02DD6EC4348DE32124138812FDAE0E...,ics20,1000000,,,,,,NaN,udvpn,sdk.coin,transfer/channel-63/transfer/channel-1,"[channel-63, channel-1]",sifchain-1,channel-83,None,None,False
10595,sentinelhub-2,ibc/CFC3F79D1E0CBBAF2C63A14C97434AFC532F5194C7...,ics20,1647596895,,,,,,NaN,udvpn,sdk.coin,transfer/channel-12/transfer/channel-141/trans...,"[channel-12, channel-141, channel-2]",cosmoshub-4,channel-186,None,None,False


### Get assetlists

In [5]:
assets_json = get_asset_json_dict(assets_df=assets_df, chain_id_name_dict=chain_id_name_dict)

In [6]:
print(f'Chains in chain-registry: {len(chain_id_name_dict.keys())}\nChain indexed: {len(assets_json.keys())}')

Chains in chain-registry: 118
Chain indexed: 107


### Validate assetlists

In [7]:
with open('assetlist.schema.json', 'r') as asset_list_schema_file:
    assetlist_schema_json = json.load(asset_list_schema_file)
for chain_id in assets_json.keys():
    validate(instance=assets_json[chain_id], schema=assetlist_schema_json)

### Write assetlists

In [8]:
for chain_id in assets_json.keys():
    print(chain_id)
    try:
        os.mkdir(path=f'data_json/{chain_id_name_dict[chain_id]}')
    except Exception as e:
        print(e)
    with open(f'data_json/{chain_id_name_dict[chain_id]}/assetlist.json', 'w') as assetlist_file:
        json.dump(obj=assets_json[chain_id], fp=assetlist_file, ensure_ascii=False, indent=4)
with open(f'data_json/all_assets.json', 'w') as all_assets_file:
    json.dump(obj=[assets_json[chain_id] for chain_id in assets_json.keys()],
              fp=all_assets_file, ensure_ascii=False, indent=4)

FUND-MainNet-2
[Errno 17] File exists: 'data_json/unification'
LumenX
[Errno 17] File exists: 'data_json/lumenx'
Oraichain
[Errno 17] File exists: 'data_json/oraichain'
ShareRing-VoyagerNet
[Errno 17] File exists: 'data_json/shareledger'
acre_9052-1
[Errno 17] File exists: 'data_json/acrechain'
agoric-3
[Errno 17] File exists: 'data_json/agoric'
aioz_168-1
[Errno 17] File exists: 'data_json/aioz'
akashnet-2
[Errno 17] File exists: 'data_json/akash'
arkh
[Errno 17] File exists: 'data_json/arkh'
axelar-dojo-1
[Errno 17] File exists: 'data_json/axelar'
beezee-1
[Errno 17] File exists: 'data_json/beezee'
bitcanna-1
[Errno 17] File exists: 'data_json/bitcanna'
bitsong-2b
[Errno 17] File exists: 'data_json/bitsong'
bluzelle-9
[Errno 17] File exists: 'data_json/bluzelle'
bostrom
[Errno 17] File exists: 'data_json/bostrom'
canto_7700-1
[Errno 17] File exists: 'data_json/canto'
carbon-1
[Errno 17] File exists: 'data_json/carbon'
cheqd-mainnet-1
[Errno 17] File exists: 'data_json/cheqd'
chihuahu